In [1]:
import os 

NUM_CPU = len(os.sched_getaffinity(0))
print(f'Number of CPUs: {NUM_CPU}')
NUM_THREADS = 1
os.environ["OMP_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["MKL_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["OPENBIAS_NUM_THREADS"] = str(NUM_THREADS)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(NUM_THREADS)
os.environ["NUMEXPR_NUM_THREADS"]  = str(NUM_THREADS)

NUM_PROCESS = int(NUM_CPU // NUM_THREADS) 
print(f'Number of PROCESSs: {NUM_PROCESS}')

Number of CPUs: 96
Number of PROCESSs: 96


In [2]:
import numpy as np
import math
import time
from copy import deepcopy
np.set_printoptions(suppress=True)
from multiprocessing import Pool

def beta_true_gen(N,p,K,gamma):
    '''
    Generate beta_true
    '''
    np.random.seed(0)  
    beta_true = np.zeros([p+1,K-1])
    for k in range(K-1):
        beta_true[:,k] = np.random.normal(size = [p+1,]) 
        beta_true[:,k] = beta_true[:,k]/np.linalg.norm(beta_true[:,k])
    beta_true[0,:] = beta_true[0,:] + gamma*np.log(N)
    return beta_true

def X_gen(N, p=5, rho=0.5):
    '''
    Generate features X
    '''
    mean = np.zeros(p)
    cov = np.zeros([p,p])
    for i in range(p):
        for j in range(i,p):
            cov[i,j]=rho**(np.abs(i-j))
    cov = cov+cov.T-np.eye(p)        
    X = np.random.multivariate_normal(mean, cov, (N,)) 
    return(X)

def get_onehot(y,baseclass = None):
    '''One-hot'''
    idx = np.squeeze(y)
    ss = len(y)
    nclass = len(np.unique(y))
    z = np.zeros([ss,nclass])
    z[np.arange(ss),idx] = 1  
    ls_class = list(np.arange(nclass))
    if baseclass is None:
        baseclass = K-1
    _ = ls_class.pop(baseclass)
    return z[:,ls_class]


def gd_multilogistic_opt_ic(x,y,K0_pval,baseclass, alpha): 
    '''GMLE (GD algorithm)'''
    ss,ncov = x.shape  
    K = len(np.unique(y)) 
    y = get_onehot(y,baseclass) 
    dist = 1.0; niter = 0
    beta0 = np.zeros([ncov*(K-1),1]) 
    alpha0 = np.log((1/K0_pval-1)/(K-1))
    beta0[np.arange(K-1)*ncov] = alpha0
    while (dist>1.0e-6) & (niter<1000):
        niter=niter+1
        beta0mat = (beta0.reshape([(K-1),ncov]).T)*1.0
        link_mu = x@beta0mat  
        prob = np.exp(link_mu);prob=prob/(1+np.sum(prob,axis = 1,keepdims=True))
        resid = y-prob
        D1=((x.T@resid/ss).T).reshape([-1,1])
        beta1 = beta0 + alpha * D1
        assert beta1.shape==(ncov*(K-1),1),'shape is wrong'
        dist = np.mean(np.abs(beta1-beta0))
        beta0 = beta1
    return beta0.reshape([(K-1),ncov]).T, dist, niter 


def mle_logistic_cpu_ic(k):
    '''PMLE'''
    np.random.seed(k)
    # Subsample
    idx_k = np.where(Y==k)[0] 
    idx_k = np.concatenate((idx_0,idx_k)) 
    x = X[idx_k]; y = Y[idx_k]
    # Optimization
    ss,ncov = x.shape  
    y = y.reshape(ss,1)
    y = 1*(y!=0) 
    dist=1.0
    niter=0
    beta0 = np.zeros([ncov,1])
    alpha0 = np.log(1/K0_pval-1)
    beta0[0] = alpha0
    
    while (dist>1.0e-6) & (niter<50):
        niter=niter+1
        link_mu = x@beta0  
        prob = np.exp(link_mu);prob=prob/(1+prob)
        resid=y-prob
        D1=x.T@resid/ss  # The first-order derivative
        weight = np.sqrt(prob*(1-prob))
        wx  = weight*x 
        del weight
        D2=wx.T@wx/ss+1.0e-6*np.eye(ncov)  # The second-order derivative
        del wx 
        step=np.linalg.inv(D2)@D1
        beta1=beta0+step
        assert beta1.shape==(ncov,1),'shape is wrong'
        dist=np.mean(np.abs(beta1-beta0))
        beta0=beta1
    return beta1.reshape([ncov,])

In [7]:
'''
N = 10**5, 2*10**5, 5*10**5
'''
N = 10**5     # Sample size
p = 500       # Feature dimension
K = 51        # Number of Classes
gamma = -0.5  # Rareness
nsimu = 100

alpha = 40
baseclass = 0
K0_pval = 0.9
a = 25

t_gd = np.zeros(nsimu)
t_pmle = np.zeros(nsimu)

beta_hat = np.zeros([nsimu,p+1,(K-1)])
beta_hat_gd = np.zeros([nsimu,p+1,(K-1)])
dist = np.zeros([nsimu]); dist_gd = 0*dist
niter = np.zeros([nsimu]); niter_gd = 0*niter
num_pos_idxs = np.zeros([nsimu,K])
beta_true = beta_true_gen(N,p,K,-0.5) 

t3 = time.time()
for b in range(nsimu):
    if (b % a==0): t1 = time.time()
    
    np.random.seed(b)
    X = X_gen(N,p)
    X = np.hstack([np.ones([N,1]),X]) 
    prob = np.exp(X@beta_true) 
    prob = np.hstack([np.ones([N,1]),prob])   
    prob = prob/np.sum(prob,1).reshape([N,1]) 
    prob = np.cumsum(prob,1) 
    Y = (np.random.uniform(size = [N,1])<prob).astype(np.int16) 
    Y = np.argmax(Y,1) 
    idx_0 = np.where(Y==0)[0] 
    
    cls = np.zeros([N,K])
    cls[np.arange(N),Y] = 1 
    num_pos_idxs[b] =  np.sum(cls,0) 
    del cls
    
    ## GMLE: GD
    t5 = time.time()
    beta_hat_gd[b], dist_gd[b], niter_gd[b] = gd_multilogistic_opt_ic(X,Y,K0_pval,baseclass,alpha)
    t6 = time.time(); t_gd[b] = t6-t5
    print(f'GMLE(GD): {np.round(t6-t5,2)}s')
    
    ## PMLE: NR
    np.random.seed(b)
    t5 = time.time()
    with Pool(K-1) as pool:
        beta_hat[b] = np.array(pool.map(mle_logistic_cpu_ic, [k for k in range(1,K)])).T
    t6 = time.time(); t_pmle[b] = t6-t5
    print(f'PMLE: {np.round(t6-t5,2)}s')
    
    if (b % a==0): 
        t2 = time.time(); print('epoch {}: {}s'.format(b,np.round(t2-t1, 2)))

t4 = time.time(); print(f'\n{np.round(t4-t3,4)}s')

# np.savez(f'/mnt/multi-class_simu/simulation/result/simu1/N{int(N/10**5)}_K{K}_p{p}_K0_pval[{K0_pval}].npz', 
#          beta_pmle=beta_hat, beta_gd=beta_hat_gd, beta_true=beta_true,
#          t_pmle = t_pmle, t_gd = t_gd, num_pos_idxs=num_pos_idxs,
#          niter_gd = niter_gd, dist_gd = dist_gd)

GMLE(GD): 523.11s
PMLE: 87.25s
epoch 0: 624.19s
GMLE(GD): 520.78s
PMLE: 86.38s
GMLE(GD): 514.15s
PMLE: 89.37s
GMLE(GD): 525.66s
PMLE: 86.35s
GMLE(GD): 513.02s
PMLE: 88.36s
GMLE(GD): 517.69s
PMLE: 83.38s
GMLE(GD): 525.66s
PMLE: 88.87s
GMLE(GD): 519.45s
PMLE: 84.52s
GMLE(GD): 519.34s
PMLE: 88.98s
GMLE(GD): 510.35s
PMLE: 85.8s
GMLE(GD): 516.94s
PMLE: 86.79s
GMLE(GD): 515.91s
PMLE: 86.94s
GMLE(GD): 519.27s
PMLE: 87.75s
GMLE(GD): 516.74s
PMLE: 86.33s
GMLE(GD): 514.99s
PMLE: 86.34s
GMLE(GD): 508.76s
PMLE: 86.34s
GMLE(GD): 500.96s
PMLE: 87.27s
GMLE(GD): 504.27s
PMLE: 84.1s
GMLE(GD): 504.52s
PMLE: 86.93s
GMLE(GD): 506.16s
PMLE: 88.87s
GMLE(GD): 518.2s
PMLE: 85.83s
GMLE(GD): 518.08s
PMLE: 87.53s
GMLE(GD): 517.88s
PMLE: 84.48s
GMLE(GD): 511.24s
PMLE: 86.34s
GMLE(GD): 497.23s
PMLE: 87.06s
GMLE(GD): 498.74s
PMLE: 84.6s
epoch 25: 597.14s
GMLE(GD): 510.31s
PMLE: 84.62s
GMLE(GD): 501.12s
PMLE: 85.02s
GMLE(GD): 505.6s
PMLE: 84.5s
GMLE(GD): 506.48s
PMLE: 86.24s
GMLE(GD): 507.38s
PMLE: 84.79s
GMLE(GD): 